# Managing textual data using pandas

This lecture introduces you to preparing and managing textual data for analysis using *[pandas](http://pandas.pydata.org/)*, a library for processing and analysing table-like data.

Typically, you must load and prepare the data yourself, either from a single file or from multiple files.

Typical formats for distributing corpora include CSV files, which stands for Comma-separated Values, and JSON, which stands for JavaScript Object Notation or simple plain text files.

After this lecture, you should:

- have a basic understanding about *pandas*, a Python library for processing and analysing data
- how to import data into a *pandas* DataFrame
- how to explore data stored in a *pandas* DataFrame
- how to append data to a *pandas* DataFrame
- how to save the data in a *pandas* DataFrame

The following example shows how to load a corpus from a CSV file for processing in Python using the [SFU Opinion and Comments Corpus (SOCC)](https://github.com/sfu-discourse-lab/SOCC).

Let's load a part of the SFU Opinion and Comments Corpus, which contains the opinion articles from [The Globe and Mail](https://www.theglobeandmail.com/), a Canadian newspaper.

## Importing data to *pandas*

Let's start by importing the *pandas* library. Note that we can control the name of the imported module using the `as` addition to the `import` command. pandas is commonly abbreviated `pd`.

In [ ]:
import pandas as pd

### Importing data from a single file

If you get lucky, your data might be contained in a single file in a structured format, such as comma-separated values (CSV).

*pandas* provides plenty of functions for [reading data in various formats](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html). You can even try importing Excel sheets – although I would not recommend this.

For current purposes, after importing *pandas* as `pd`, we can use the `read_csv()` functions to read files with comma-separated values, such as the SOCC corpus.

In [ ]:
socc = pd.read_csv('00_data/socc_gnm_articles.csv')

*pandas* does all the heavy lifting for us and returns the contents of the CSV file in a *pandas* DataFrame, which is data structure native to *pandas*.

In [ ]:
type(socc)

Let's use the `head()` method of a DataFrame to check out the first five rows in the DataFrame.

In [ ]:
socc.head(5)

As you can see, the data is organised into a tabular form: a DataFrame, the fundamental data structure in *pandas*.

The DataFrame contains several columns such as **article_id**, **title** and **article_text**, accompanied by an index for each row (**0, 1, 2, 3, 4**).

The `.at[]` accessor can be used to inspect a single item in the DataFrame.

Let's examine the value in the column **title** at index 123.

In [ ]:
socc.at[123, 'title']

### Importing data from multiple files

Another common scenario is that you have multiple files with text data, which you want to load into *pandas*.

Let's first collect the files that we want to load.

In [ ]:
# Import the patch library
from pathlib import Path

# Create a Path to the corpus directory
corpus_dir = Path('00_data')

# Get all .txt files in the corpus directory
corpus_files = list(corpus_dir.glob('*.txt'))

To accommodate our data, let's create an empty pandas DataFrame and specify its shape in advance, that is, the number of rows (`index`) and the names of the columns `columns`.

We can easily get the number of rows needed using Python's `range()` function, which generates a list of numbers that fall within certain range.

Here we start from zero and finish when we get up to the number of files in the corpus.

For columns, we simply create columns for holding the name of the file and its actual contents.

In [ ]:
df = pd.DataFrame(index=range(0, len(corpus_files)), columns=['filename', 'text'])

df

Now that we have an empty data with rows for each file in the corpus, we can loop over the files and add their contents to the DataFrame.

In [ ]:
# Loop over the corpus files
for i, f in enumerate(corpus_files):
    
    # Open the file for reading
    c_file = open(f, encoding="utf-8")
    
    # Get the filename from the Path object
    filename = f.name
        
    # Read the file contents
    text = c_file.read()
    
    # Assign the text from the file to index 'i' at column 'text'
    # using the .at accessor – note that this modifies the DataFrame
    # "in place" – you don't need to assign the result into a variable
    df.at[i, 'text'] = text
    
    # We then do the same to the filename
    df.at[i, 'filename'] = filename

Let's check the result.

In [ ]:
df

### Quick in-class exercise

Let's go back to the SOCC corpus stored under `socc`.

Who is the author (`author`) of article at index 256? 

How many top-level comments (`ntop_level_comments`) did the article at index 1000 receive?

In [ ]:
### Enter your code below and press Shift+Enter to run the cell


*pandas* also provides various methods for examining the contents of entire columns, which can be accessed just like the keys and values of a Python dictionary.

In [ ]:
socc['author']

As you can see, the column `author` contains 10399 objects. 

The numbers on the left-hand side give the index: you can think of them as row numbers.

We can use built-in methods such as `value_counts()` to easily count the number of unique authors in the corpus.

In [ ]:
socc['author'].value_counts()

Looks like the editorial team is responsible for most of the editorials. Who would have guessed?

Let's take another look at the data by visualising the result by calling the `.plot()` method!

In [ ]:
# This is some Jupyter magic that allows us to render matplotlib plots in the notebooks!
# You only need to enter this command once.
%matplotlib inline

# Count the values in the column 'author' and clip the result to top-10 before plotting.
socc['author'].value_counts()[:10].plot(kind='bar')

For columns with numerical values, we can also use the `.describe()` method to get basic descriptive statistics on the data.

In [ ]:
socc['ntop_level_comments'].describe()

As we can see, the column `ntop_level_comments` has a total of 10339 rows. 

The average number of comments received by an editorial is approximately 26, but this number fluctuates, as the standard deviation from the mean is nearly 40.

- Some editorials do not have any comments at all, as indicated by the minimum value of 0. 
- The lowest quartile shows that 25% of the data has only one comment or less (none).
- The second quartile (50%), which is also known as the median, indicates that *half* of the data has less than 14 comments and *half* has more than 14 comments. 
- The third quartile shows that 75% of the data has 35 comments or less. 
- The most commented editorial has 1378 comments.

What if we would like to find the articles that received no comments at all?

For this purpose, we can use the *pandas* accessor `.loc` to access columns or rows based on their labels.

We know that the number of comments is stored in the column `ntop_level_comments`, but we also need to tell *pandas* which DataFrame contains this column (`socc`). This causes the somewhat repetitive expression in the beginning.

Next, we need to provide a command for "is equal to". Since the single equal sign `=` is reserved for assigning variables, two equal signs are used for comparison.

Finally, we place the value we want to evaluate against on the right-hand side of the double equal sign `==`.

In [ ]:
socc.loc[socc['ntop_level_comments'] == 0]

We can also combine multiple criteria using the `&` symbol, which is the Python operator for "AND".

Note that individual criteria must be placed in parentheses `()` to perform the operation.

Let's check if the first author in our result, Hayden King, wrote any other articles with zero comments.

In [ ]:
socc.loc[(socc['ntop_level_comments'] == 0) & (socc['author'] == 'Hayden King')]

### Quick in-class exercise

How many articles with zero top-level comments were authored by the editorial team (`GLOBE EDITORIAL`)?

Write out the whole command yourself instead of copy-pasting to get an idea of the syntax.

In [ ]:
### Enter your code below and press Shift+Enter to run the cell


## Adding information to pandas DataFrames

You can easily add information to *pandas* DataFrames.

One common scenario could involve loading some data from an external file (such as a CSV or JSON file), performing some analyses and storing the results to the same DataFrame.

We can easily add an empty column to the DataFrame. This is achieved using the column accessor `[]` and the Python datatype `None`.

Let's add a new column named `r_comments` to the DataFrame `socc`.

In [ ]:
socc['r_comments'] = None

In [ ]:
socc.head(5)

Let's populate the column with some data by calculating how many percent of the comments are top-level comments.

We can assume that a high percentage of top-level comments indicates comments about the article, whereas a lower percentage indicates more discussion about the comments posted.

To get the proportion of top-level comments out of all comments, we must divide the number of top-level comments by the number of all comments.

In [ ]:
socc['r_comments'] = socc['ntop_level_comments'] / socc['ncomments']

As you can see, *pandas* column accessors can be used very flexibly to access and manipulate (in this case, divide) the data.

In [ ]:
socc.head(5)

As you can see, the column `r_comments` now stores the result of our calculation!

However, we should also keep in mind that some articles did not receive any comments at all: thus we would have divided zero by zero.

Let's take a look at these cases at the tail of the DataFrame.

In [ ]:
socc.tail(5)

As you can see, the result is marked as `NaN` or "not a number".

This indicates that the operation was performed on these cells as well, but the result was not a number.

*pandas* automatically ignores these values when performing calculations, as show by the `.describe()` method.

In [ ]:
socc['r_comments'].describe()

Note the difference in the result for the count. Only 7797 items out of 10399 were included in the calculation.

What if we would like to do some NLP and store the results in the DataFrame?

Instead of processing the whole DataFrame, let's select articles that seem to cause discussion among commenters and have more than 200 comments.

In [ ]:
talk = socc.loc[(socc['r_comments'] <= 0.384) & (socc['ncomments'] >= 200)]

In [ ]:
talk

Let's import spacy and load the language model for English and assign it to the variable `nlp`.

In [ ]:
import spacy

nlp = spacy.load('en_core_web_md')  # Note that we now load a medium-sized language model!

Assuming we want to process the article titles, let's break down the process into small steps and create a placeholder column named `title_nlp` first.

In [ ]:
talk['title_nlp'] = None

*pandas* warns about performing this command, because `talk` is only a slice or a _view_ into the DataFrame. 

Assigning a new column to **only a part of the DataFrame** would cause problems by breaking the tabular structure.

We can fix the situation by creating a _deep copy_ of the slice using Python's `.copy()` method.

In [ ]:
talk = talk.copy()

Let's try creating an empty column again.

In [ ]:
talk['title_nlp'] = None

In [ ]:
talk

Feeding each title for our NLP pipeline is not as straightforward as dividing one pandas column full numbers with another of the same type.

To take each individual article title and feed it to spaCy for processing, we need to use the pandas `.apply()` method with a `lambda` function.

Python's `lambda` functions are essentially functions that are defined on the fly.

The `lambda` function is followed by a variable, in this case `title_text` and a colon `:`.

What `lambda` essentially does is somewhat equivalent to "take whatever is given and do something with it".

The part on the right-hand side of the colon `:` tells Python to take the column contents provided by `.apply()` and feed them to spaCy for processing.

In [ ]:
talk['title_nlp'] = talk['title'].apply(lambda title_text: nlp(title_text))

In [ ]:
talk

We now have the processed titles in a separate column named `title_nlp`!

Let's take a closer look at the first row, whose index is 2.

In [ ]:
talk.at[2, 'title_nlp']

In [ ]:
type(talk.at[2, 'title_nlp'])

As you can see, the cell contains a spaCy _Doc_ object.

Let's now define our own Python **function** to fetch lemmas for each noun in the title.

Python functions are defined using the command `def`, which is followed by the name of the function, in this case `get_nouns`. The input to the function is given in parentheses that follow the name of the function.

In this case, we name a variable for the input called `nlp_text`. This is an arbitrary variable, which is needed for referring to whatever is being provided as input to the function. To put it simply, you can think of this variable as the input. 

In [ ]:
def get_nouns(nlp_text):
    
    # First we make sure that the input is of correct type
    assert type(nlp_text) == spacy.tokens.doc.Doc
    
    # Let's set up a placeholder list for our lemmas
    lemmas = []
    
    # We begin then begin looping over the Doc object
    for token in nlp_text:
        
        # If the fine-grained POS tag for the token is a noun (NN)
        if token.tag_ == 'NN':
            
            # Append the token lemma to the list of lemmas
            lemmas.append(token.lemma_)
            
    # When the loop is complete (note the indentation), 
    # join the list of items together and return a string
    return lemmas

Now that we have defined our function, let's use it to collect all nouns to the column `nouns`.

In [ ]:
talk['nouns'] = talk['title_nlp'].apply(lambda text: get_nouns(text))

In [ ]:
talk

We can also easily extract information from pandas DataFrames into Python's native data structures. 

The `tolist()` method, for instance, can be used to extract the column contents into a list.

In [ ]:
noun_list = talk['nouns'].tolist()

noun_list

What we have now is a list of lists. Let's loop over the list and collect the items into a single list named `final_list`.

In [ ]:
# Set up the placeholder list
final_list = []

# Loop over each list in the list of lists
for nlist in noun_list:
    
    # Extend the final list with the current list
    final_list.extend(nlist)

Let's briefly examine the first ten items in final list and then count the number of items in the list.

In [ ]:
final_list[:10]

In [ ]:
len(final_list)

We can also easily count the number of unique items in the list using a Python *Counter* object and its `most_common()` method.

In [ ]:
from collections import Counter

Counter(final_list).most_common()

What do you think? Is there a connection between editorial topics and the ensuing discussion among readers?

## Saving the results

*pandas* DataFrames can be easily saved as *pickled* objects using the `to_pickle()` method.

The `to_pickle()` method takes a string as input, which defines a path to the file in which the DataFrame should be written.

Let's pickle the DataFrame with the three articles stored under `df` into a file named `pickled_df.pkl` into the directory `00_data`.

In [ ]:
df.to_pickle('00_data/pickled_df.pkl')

We can easily check if the data has been saved successfully by reading the file contents using the `read_pickle()` method.

In [ ]:
df_2 = pd.read_pickle('00_data/pickled_df.pkl')

df_2

Let's compare the DataFrames, which returns a Boolean value (True/False) for each cell.

In [ ]:
df == df_2

## Wrapping up

This lecture should have given you a brief introduction to *pandas* and its basic data structure, a DataFrame.